# Setup

In [494]:
%pip install imageio
%pip install pyvirtualdisplay
%pip install tf-agents
%pip install pyglet
%pip install matplotlib
%pip install tensorflow



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you 

In [495]:

import tensorflow as tf
import numpy as np
from keras.callbacks import TensorBoard
import time
import random
import os
import csv
import array


##########################################################################################
# PPO Imports
##########################################################################################
import tensorflow as tf

from tf_agents.agents.ppo import ppo_clip_agent
from tf_agents.specs import tensor_spec
from tf_agents.specs import array_spec
from tf_agents.specs import BoundedTensorSpec
from tf_agents.specs import TensorSpec
from tf_agents.trajectories import time_step as ts
from tf_agents.environments import tf_py_environment



import tensorflow_probability as tfp

from tf_agents.networks import value_network
from tf_agents.networks import actor_distribution_network
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.replay_buffers import py_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

from tf_agents.environments import py_environment

PPOClipped

In [496]:
GLOBAL_EPSILON = 0.2
GLOBAL_EPOCHS = 3

# PPO Agent 
class PPOClipped:

    def __init__(self, env):
        # Constants
        self.actor_fc_layers = (64,64)
        self.value_fc_layers = (64,64)
        self.epsilon = GLOBAL_EPSILON
        self.epochs = GLOBAL_EPOCHS

        self.time_step_tensor_spec = tensor_spec.from_spec(env.time_step_spec())
        self.observation_tensor_spec = tensor_spec.from_spec(env.observation_spec())
        self.action_tensor_spec = tensor_spec.from_spec(env.action_spec())

        # Building the PPO Agent
        self.actor_net = self.createActorNet()
        self.value_net = self.createValueNet()
        self.optimizer = self.createOptimizer()
        self.train_step_counter = tf.Variable(0)

        self.ppo_agent = self.createPPOAgent()

        self.batch_size = 64
        self.replay_buffer = self.createReplayBuffer()
        self.iterator = self.createBufferIterator()


    def createActorNet(self):
        actor_net = actor_distribution_network.ActorDistributionNetwork(
            self.observation_tensor_spec,
            self.action_tensor_spec,
            fc_layer_params=self.actor_fc_layers,
            activation_fn=tf.keras.activations.tanh,
            kernel_initializer=tf.keras.initializers.Orthogonal(),
            seed_stream_class=tfp.util.SeedStream
        )
        return actor_net

    def createValueNet(self):
        value_net = value_network.ValueNetwork(
            self.observation_tensor_spec,
            fc_layer_params=self.value_fc_layers,
            activation_fn=tf.keras.activations.tanh,
            kernel_initializer=tf.keras.initializers.Orthogonal()
        )
        return value_net

    def createOptimizer(self):
        initial_learning_rate = 1e-3
        decay_steps = 1000
        decay_rate = 0.96
        learning_rate = tf.optimizers.schedules.ExponentialDecay(
            initial_learning_rate, decay_steps, decay_rate, staircase=True
        )
        optimizer = tf.optimizers.Adam(learning_rate=learning_rate)
        learning_rate = 2.5e-4
        optimizer = tf.optimizers.Adam(learning_rate=learning_rate)
        return optimizer


    def createPPOAgent(self):
        agent_ppo = ppo_clip_agent.PPOClipAgent(
            time_step_spec=self.time_step_tensor_spec,
            action_spec=self.action_tensor_spec,
            optimizer=self.optimizer,
            normalize_observations=True,
            normalize_rewards=True,
            actor_net=self.actor_net,
            value_net=self.value_net,
            importance_ratio_clipping=self.epsilon,
            num_epochs=self.epochs,
            use_gae=True,
            train_step_counter=self.train_step_counter,
        )
        agent_ppo.initialize()
        agent_ppo.train_step_counter.assign(0)
        # (Optional) Optimize by wrapping some of this code in a graph using TF function.
        agent_ppo.train = common.function(agent_ppo.train)
        return agent_ppo

    def createReplayBuffer(self):
        replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
            data_spec= self.ppo_agent.collect_policy.trajectory_spec,
            # self.ppo_agent.policy.trajectory_spec,
            batch_size=1,
            max_length=10000)
        return replay_buffer

    def addToBuffer(self, traj):
        self.replay_buffer.add_batch(traj)
        
    def createBufferIterator(self):
        n_step_update = 10
        # batch_size = 
        batch_size = self.batch_size
        dataset = self.replay_buffer.as_dataset(
            num_parallel_calls=3, sample_batch_size=batch_size,
            num_steps=n_step_update + 1).prefetch(batch_size)
        iterator = iter(dataset)
        return iterator

    def train(self):
        experience, unused_info = next(self.iterator)
        self.ppo_agent.train(experience)
        print('Step Counter: {0}'.format(self.train_step_counter))

    def getAction(self, time_step):
        # return self.ppo_agent.policy.action(time_step)
        return self.ppo_agent.collect_policy.action(time_step)

MqEnvironment

In [497]:
class MqEnvironment(py_environment.PyEnvironment):
    
    def __init__(self, maxqos, minqos):
        self._observation_spec = TensorSpec(shape=(8,), dtype=tf.float32, name='observation')
        self._action_spec = BoundedTensorSpec(
            shape=(), dtype=tf.int32, minimum=-1, maximum=1, name='action')
        self._reward_spec = TensorSpec(shape=(), dtype=tf.float32, name='reward')

        self._maxqos = maxqos
        self._minqos = minqos

        self._rewards = 0
        self._current_time_step = None
        self._action = None

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec
    
    def reward_spec(self):
        return self._reward_spec
    
    def _reset(self):
        observation_zeros = tf.zeros((8,), dtype=tf.float32)
        reward = tf.convert_to_tensor(1, dtype=tf.float32)
        self._current_time_step = ts.transition(observation_zeros, reward=reward, discount=1.0)
        
        return self._current_time_step
    
    def _step(self, action):
        self._action = action
        return self._current_time_step

    def mq_step(self, action, state):
        observation = tf.convert_to_tensor(state, dtype=tf.float32)
        reward = self.get_reward(observation)
        self._current_time_step = ts.transition(observation, reward=reward, discount=1.0)
        self._action = action
        return self._current_time_step
    
    def get_reward(self, observation):
        global_avg_spark, thpt_var, cDELAY, cTIMEP, RecSparkTotal, RecMQTotal, state, mem_use = observation.numpy()
        lst_thpt_glo, lst_thpt_var, lst_cDELAY, lst_cTIMEP, lst_RecSparkTotal, lst_RecMQTotal, lst_state, lst_mem_use = self.current_time_step().observation.numpy()
        r_thpt_glo, r_thpt_var, r_cDELAY, r_cTIMEP, r_RecSparkTotal, r_RecMQTotal, r_state, r_mem_use = np.zeros(8, dtype=np.float32)
        
        if lst_mem_use < self._minqos:
            r_mem_use = -10
        elif lst_mem_use > self._maxqos:
            r_mem_use = -10
        elif mem_use > lst_mem_use:
            r_mem_use = 10
        r_thpt_var = thpt_var
        reward = r_thpt_glo + r_thpt_var + r_cDELAY + r_cTIMEP + r_RecSparkTotal + r_RecMQTotal + r_state + r_mem_use
        
        if reward < 0:
            reward = 0.0
        return tf.convert_to_tensor(reward, dtype=tf.float32)

PPOAgentMQ

In [500]:
class PPOAgentMQ:
    def __init__(self, start_state, upper_limit, lower_limit):
        # self.env = tf_py_environment.TFPyEnvironment(MqEnvironment(upper_limit, lower_limit))
        self.env = MqEnvironment(upper_limit, lower_limit)
        self.ppo_agent = PPOClipped(self.env)
        self.buffer = False

        # TimeStep of the environment
        self._last_state = self.env.reset()
        self._last_action = self.ppo_agent.getAction(self._last_state)
        self._batch_size = self.ppo_agent.batch_size
        # self.env._set_action(self._last_action)
        self.env._current_action = self._last_action
        self._last_reward = None
        self._first_exec = True
        
    def step(self, _new_state):
      
        new_state = tf.convert_to_tensor(_new_state, dtype=tf.float32)
        # Get the current state from the environment
        last_time_step = self.env.current_time_step()
        last_action = self._last_action
        # From new state, calculate reward and change environment state
        current_time_step = self.env.mq_step(last_action, new_state)
        
        traj = trajectory.from_transition(last_time_step, last_action, current_time_step)
        traj_batched = tf.nest.map_structure(lambda t: tf.stack([t] * 1), traj)
        self.ppo_agent.addToBuffer(traj_batched)
        
        if self.ppo_agent.replay_buffer.num_frames().numpy() > self._batch_size:
            # for _ in range(64):
            #     self.ppo_agent.addToBuffer(traj_batched)
            # self._first_exec = False
            # 
            self.ppo_agent.train()
        
        # -----------------------------

        # Collect initial data
        # initial_time_step = train_env.reset()
        # while not initial_time_step.is_last():
        #     action_step = agent.collect_policy.action(initial_time_step)
        #     next_time_step = train_env.step(action_step.action)
        #     traj = trajectory.from_transition(initial_time_step, action_step, next_time_step)
        #     replay_buffer.add_batch(traj)
        #     initial_time_step = next_time_step

        return 0


    def finish(self, last_state):
        pass
        # with open(CSVFILENAME, mode='a+') as episode_file:
        #     writer = csv.writer(episode_file, delimiter=',')
        # 
        #     writer.writerow([self.episode, self.epsilon, mean_squared_reward])

RUN Env Simulation

In [502]:
# Constants
sim_iterations = 70
# ---------------

maxqos = 1000
minqos = 10

start_state = np.zeros((8,)).tolist()

agent = PPOAgentMQ(start_state, maxqos, minqos)

for _ in range(sim_iterations):
    state = np.random.rand(8).astype(np.dtype('float32')).tolist()
    action = agent.step(state)


last_state = [0,0,0,0,0,0,0,0]
agent.finish(last_state)

Step Counter: <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=3>
Step Counter: <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=6>
Step Counter: <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=9>
Step Counter: <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=12>
Step Counter: <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=15>
Step Counter: <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=18>


In [ ]:
for _ in range(3):
    i =next(agent.ppo_agent.iterator)
    print(i)


In [ ]:


start_state = np.zeros((8,)).tolist()

env = MqEnvironment(minqos, maxqos)
# env = tf_py_environment.TFPyEnvironment(MqEnvironment(maxqos, minqos))
pa = PPOClipped(env)
mq = PPOAgentMQ(start_state, maxqos, minqos)

# pa.getAction(env.reset()).action.numpy()
# pa.getAction(env.reset())
print(pa.ppo_agent.collect_policy.policy_step_spec)
print(pa.ppo_agent.policy.policy_step_spec)

# 
# print(env.action_spec())
# print(pa.action_tensor_spec)
# 
# print(env.reset())
# print(pa.getAction(env.reset()))

In [ ]:
state = np.zeros((8,)).tolist()
qosmax = 1000
qosmin = 10
# ppomq = PPOAgentMQ(state, qosmax, qosmin)
# 
# ppomq.ppo_agent.ppo_agent.collect_policy.time_step_spec 
# mts=tf.convert_to_tensor([0,0,0,0,0,0,0,0], dtype=np.dtype('float32'))
# time_step = ts.transition(mts, reward=tf.convert_to_tensor(1, dtype=np.dtype('int32')), discount=1.0)
# action = ppomq.ppo_agent.getAction(time_step)
# 
# ppomq.env._current_timestep = time_step
# ppomq.env.step(action, time_step)

# ppomq.ppo_agent.ppo_agent.policy.action(ts.transition(mts, reward=tf.convert_to_tensor(1, dtype=np.dtype('int32')), discount=1.0))
# tf.convert_to_tensor([0,0,0,0,0,0,0,0]).numpy().tolist()
# tf.convert_to_tensor([0,0,0,0,0,0,0,0], dtype=np.dtype('float32'))

# mqenv = tf_py_environment.TFPyEnvironment(MqEnvironment(10,1000))
# mqenv.reset()
# ts = mqenv.reset()

# mqenv.current_time_step()[3].numpy()[0].tolist()
# tf.convert_to_tensor(state, dtype=np.dtype('float32'))


env = MqEnvironment(qosmax, qosmin)
env.action_spec()
# ppomq = PPOAgentMQ(state, qosmax, qosmin)
time_step_tensor_spec = env.time_step_spec()
observation_tensor_spec = env.observation_spec()
action_tensor_spec = env.action_spec()

time_step_tensor_spec
observation_tensor_spec
action_tensor_spec

# actor_distribution_network.ActorDistributionNetwork(
#     observation_tensor_spec,
#     action_tensor_spec,
#     fc_layer_params=(64,64),
#     activation_fn=tf.keras.activations.tanh,
#     kernel_initializer=tf.keras.initializers.Orthogonal(),
#     seed_stream_class=tfp.util.SeedStream
# )

print(tensor_spec.from_spec(observation_tensor_spec))
print(tf.convert_to_tensor(state))
# print(tf.convert_to_tensor(state, dtype=tf.int32))

# r = tf.convert_to_tensor(state, dtype=tf.float32)
# r

In [ ]:

# print(tensor_spec.from_spec(observation_tensor_spec))
# print(tf.convert_to_tensor(state))
# print(tf.convert_to_tensor(state, dtype=tf.int32))

# r = tf.convert_to_tensor(state, dtype=tf.float32)

# obs_tensor_spec = tensor_spec.from_spec(observation_tensor_spec)
# obs_tensor = tf.convert_to_tensor(np.zeros((8,)).tolist())

# obs_tensor_spec
# obs_tensor
# state = np.zeros((8,)).tolist()
# reward = 100
# current_time_step = ts.transition(state, reward=reward, discount=1.0)
# print(current_time_step)
# current_time_step = ts.transition(current_time_step.observation, reward=100, discount=1.0)
# current_time_step